In [25]:
import pandas as pd
import numpy as np
import os

In [26]:
data = pd.read_excel('routine.xlsx', sheet_name=2 ,skiprows=4)
data.columns = data.columns.str.replace('\n', '_')
data.columns = data.columns.str.replace(':', '')
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace(r'\.\d+', '', regex=True)
df = data.fillna('-') 
df.columns = df.columns.str.upper()

skips the unnecessary sheets and also skipps the first 4 rows which r the headings of the sheet.

In [27]:
data_cleaning = df.drop(["CLASS_SIZE", "ASSIGNED_HALL", "HALL_CAPACITY_MATCH", "UNNAMED_57"], axis='columns')
# len(data.columns)
indexList = list(range(61))
for i in range(2, 61, 5):
    indexList.remove(i)
clean_data = data_cleaning.iloc[:, indexList]

adds the day in all the rows of the day column 

In [28]:
def fill_days(column):
    current_day = None
    new_column = column.copy()  # Create a copy to avoid warnings
    for i, value in enumerate(new_column):
        if pd.notna(value) and value != '-':
            current_day = value
        elif value == '-' and current_day is not None:
            new_column.iloc[i] = current_day
    return new_column

# Make sure clean_data is a copy, not a view
clean_data = clean_data.copy()

# Apply the function to the 'DAY' column
clean_data.loc[:, 'DAY'] = fill_days(clean_data['DAY'])

this fills in all the columns in day secction. all the columns r filled wit day name

In [29]:
days_of_week = ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY']
monday_df, tuesday_df, wednesday_df, thursday_df, friday_df, saturday_df = [clean_data.loc[clean_data['DAY'] == day] for day in days_of_week]

# Create a new folder to store the CSV files
new_folder = "excel data"
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

for day in days_of_week:
    day_data = clean_data[clean_data['DAY'] == day]
    filename = f"{day}.xlsx"  # Create a unique filename for each day
    file_path = os.path.join(new_folder, filename)  # Join the folder path with the filename
    day_data.to_excel(file_path, index=False)


makes indivigual dataframes for every 'day' section of the week